In [28]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import (cosine_similarity,
                                     euclidean_distances,
                                     cosine_distances,
                                     pairwise_distances)

from thefuzz import process
import fuzzyset

from scipy import sparse

#product based recommender


#loading movie titles and their unique ids
movie_ids = pd.read_csv('./data/movies_with_review_id.csv')
movie_ids = movie_ids[['tconst', 'primary_title_x','movieId']]
#loading movie titles with year
movie_years = pd.read_csv('./data/movie_ids_and_year.csv')
movie_years = movie_years[['tconst', 'full_title']]
#merging the full title from movie_years on tconst
movie_ids = pd.merge(movie_ids, movie_years, on='tconst', how='left')
movie_ids.rename(columns={'primary_title_x':'primary_title'}, inplace=True)
#loading the cosine distance dataframe for recommendations
cos_df = pd.read_csv('./data/product_based_recommender_df.csv')

#creating the list of titles for fuzzy matching
title_list = fuzzyset.FuzzySet(list(movie_ids['full_title']))

#creating a function that returns the intersection of two lists
def intersection(list1, list2):
    return [item for item in list1 if item in list2]

#building the recommender

def new_user_recommender():
    correct = 'n'
    
    while correct != 'y':
        
        #asking the user to input films they like
        liked_films = input("Please enter 3 film titles that you enjoy, \nseparated by forward slashes (/):\n")

        if len(liked_films.split('/')) == 3:
            #getting the matching film titles from our list of films
            film1 = liked_films.split('/')[0]
            film2 = liked_films.split('/')[1]
            film3 = liked_films.split('/')[2]

            #appending the matching titles to a list
            liked_films = []
            liked_films.append(title_list.get(film1)[0][1])
            liked_films.append(title_list.get(film2)[0][1])
            liked_films.append(title_list.get(film3)[0][1])

            #printing the list of films for validation
            print('')
            for film in liked_films:
                print(film)

            #asking the user to validate the selected list of films
            correct = input("Please verify that the above titles are correct y/n:\n")
        
        else:
            correct = 'n'
            print('Error, please try again')
        
#creating the list of suggestions for the three selected titles

    #getting the unique ids for the liked movies:
    liked_films_tconst = []
    for film in liked_films:
        liked_films_tconst.append(\
            movie_ids[movie_ids['full_title'] == film]['tconst'].values[0])
    
    #getting the top 20 recommended films for each liked movie
    rec_list_1 = list(cos_df.sort_values(liked_films_tconst[0], ascending=False)\
                      ['tconst'][1:21])
    rec_list_2 = list(cos_df.sort_values(liked_films_tconst[1], ascending=False)\
                      ['tconst'][1:21])
    rec_list_3 = list(cos_df.sort_values(liked_films_tconst[2], ascending=False)\
                      ['tconst'][1:21])
    
    #translating the tconst back to titles
    rec_list_1_titles = []
    rec_list_2_titles = []
    rec_list_3_titles = []
    
    for tconst in rec_list_1:
        rec_list_1_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_2:
        rec_list_2_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_3:
        rec_list_3_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
    
    #if there are films that are recommended based on more than 1 liked movie
    #prioritize these films
    final_recs = intersection(rec_list_1_titles, rec_list_2_titles)
    final_recs = intersection(final_recs, rec_list_3_titles)
    
    #only want to return 5 recommendations
    for i in range(0, 5):
        final_recs.extend([rec_list_1_titles[i],\
                            rec_list_2_titles[i],\
                            rec_list_3_titles[i]])
           
    print(final_recs[0], '\n', 
          final_recs[1], '\n',
          final_recs[2], '\n',
          final_recs[3], '\n',
          final_recs[4]
         )
    
    #asking the user to "rate" the films that we have recommended to see if we're doing well
    bad_format = 'y'
    
    while bad_format == 'y':
        film_reviews = input('Please rate the above films on how likely you are to watch them (1-5). Separate the scores with commas, \n')
        film_reviews = film_reviews.replace(' ','').split(',')
        for i in range(0, len(film_reviews)):
            film_reviews[i] = int(film_reviews[i])
        
         #checking to see if they have input their scores correctly
        if all(isinstance(x, int) for x in film_reviews) == True\
        and len(film_reviews) == 5\
        and min(film_reviews) >= 1\
        and max(film_reviews) <= 5:
            print('Thank You!')
            break
        else:
            bad_format = 'y'
            
    #saving their review scores to the reviews data frame for review based recs


In [40]:
#review based recommender

#importing relevant packages
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import (cosine_similarity,
                                     euclidean_distances,
                                     cosine_distances,
                                     pairwise_distances)

from thefuzz import process
import fuzzyset

from scipy import sparse

#loading movie titles and their unique ids
movie_ids = pd.read_csv('./data/movies_with_review_id.csv')
movie_ids = movie_ids[['tconst', 'primary_title_x','movieId']]
#loading movie titles with year
movie_years = pd.read_csv('./data/movie_ids_and_year.csv')
movie_years = movie_years[['tconst', 'full_title']]
#merging the full title from movie_years on tconst
movie_ids = pd.merge(movie_ids, movie_years, on='tconst', how='left')
movie_ids.rename(columns={'primary_title_x':'primary_title'}, inplace=True)
#loading the review based recommender dataframe
review_recommender = pd.read_csv('./data/review_based_recommender_df.csv')

#building the review based recommender


#creating the list of titles for fuzzy matching
title_list = fuzzyset.FuzzySet(list(movie_ids['full_title']))

#creating a function that returns the intersection of two lists
def intersection(list1, list2):
    return [item for item in list1 if item in list2]

#building the recommender

def new_user_recommender():
    correct = 'n'
    
    while correct != 'y':
        
        #asking the user to input films they like
        liked_films = input("Please enter 3 film titles that you enjoy, \nseparated by forward slashes (/):\n")

        if len(liked_films.split('/')) == 3:
            #getting the matching film titles from our list of films
            film1 = liked_films.split('/')[0]
            film2 = liked_films.split('/')[1]
            film3 = liked_films.split('/')[2]

            #appending the matching titles to a list
            liked_films = []
            liked_films.append(title_list.get(film1)[0][1])
            liked_films.append(title_list.get(film2)[0][1])
            liked_films.append(title_list.get(film3)[0][1])

            #printing the list of films for validation
            print('')
            for film in liked_films:
                print(film)

            #asking the user to validate the selected list of films
            correct = input("Please verify that the above titles are correct y/n:\n")
        
        else:
            correct = 'n'
            print('Error, please try again')
            
#creating the list of suggestions for the three selected titles

    #getting the unique ids for the liked movies:
    liked_films_tconst = []
    for film in liked_films:
        liked_films_tconst.append(\
            movie_ids[movie_ids['full_title'] == film]['tconst'].values[0])
    
    #getting the top 20 recommended films for each liked movie
    rec_list_1 = list(review_recommender.sort_values(\
                                liked_films_tconst[0])['tconst'][1:21])
    rec_list_2 = list(review_recommender.sort_values(\
                                liked_films_tconst[1])['tconst'][1:21])
    rec_list_3 = list(review_recommender.sort_values(\
                                liked_films_tconst[2])['tconst'][1:21])
    
    #translating the tconst back to titles
    rec_list_1_titles = []
    rec_list_2_titles = []
    rec_list_3_titles = []
    
    for tconst in rec_list_1:
        rec_list_1_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_2:
        rec_list_2_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_3:
        rec_list_3_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
    
    #if there are films that are recommended based on more than 1 liked movie
    #prioritize these films
    final_recs = intersection(rec_list_1_titles, rec_list_2_titles)
    final_recs = intersection(final_recs, rec_list_3_titles)
    
    #only want to return 5 recommendations
    for i in range(0, 5):
        final_recs.extend([rec_list_1_titles[i],\
                            rec_list_2_titles[i],\
                            rec_list_3_titles[i]])
           
    print(final_recs[0], '\n', 
          final_recs[1], '\n',
          final_recs[2], '\n',
          final_recs[3], '\n',
          final_recs[4]
         )
    
    #asking the user to "rate" the films that we have recommended to see if we're doing well
    bad_format = 'y'
    
    while bad_format == 'y':
        film_reviews = input('Please rate the above films on how likely you are to watch them (1-5). Separate the scores with commas, \n')
        film_reviews = film_reviews.replace(' ','').split(',')
        for i in range(0, len(film_reviews)):
            film_reviews[i] = int(film_reviews[i])
        
         #checking to see if they have input their scores correctly
        if all(isinstance(x, int) for x in film_reviews) == True\
        and len(film_reviews) == 5\
        and min(film_reviews) >= 1\
        and max(film_reviews) <= 5:
            print('Thank You!')
            break
        else:
            bad_format = 'y'

            


In [6]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import (cosine_similarity,
                                     euclidean_distances,
                                     cosine_distances,
                                     pairwise_distances)

from thefuzz import process
import fuzzyset

from scipy import sparse

#product based recommender


#loading movie titles and their unique ids
movie_ids = pd.read_csv('./data/movies_with_review_id.csv')
movie_ids = movie_ids[['tconst', 'primary_title_x','movieId']]
#loading movie titles with year
movie_years = pd.read_csv('./data/movie_ids_and_year.csv')
movie_years = movie_years[['tconst', 'full_title']]
#merging the full title from movie_years on tconst
movie_ids = pd.merge(movie_ids, movie_years, on='tconst', how='left')
movie_ids.rename(columns={'primary_title_x':'primary_title'}, inplace=True)
#loading the cosine distance dataframe for recommendations
cos_df = pd.read_csv('./data/product_based_recommender_df.csv')
#loading the review based recommender dataframe
review_recommender = pd.read_csv('./data/review_based_recommender_df.csv')

#creating the list of titles for fuzzy matching
title_list = fuzzyset.FuzzySet(list(movie_ids['full_title']))

#creating a function that returns the intersection of two lists
def intersection(list1, list2):
    return [item for item in list1 if item in list2]

#building the recommender

def new_user_recommender():
    correct = 'n'
    
    while correct != 'y':
        
        #asking the user to input films they like
        liked_films = input("Please enter 3 film titles that you enjoy, \nseparated by forward slashes (/):\n")

        if len(liked_films.split('/')) == 3:
            #getting the matching film titles from our list of films
            film1 = liked_films.split('/')[0]
            film2 = liked_films.split('/')[1]
            film3 = liked_films.split('/')[2]

            #appending the matching titles to a list
            liked_films = []
            liked_films.append(title_list.get(film1)[0][1])
            liked_films.append(title_list.get(film2)[0][1])
            liked_films.append(title_list.get(film3)[0][1])

            #printing the list of films for validation
            print('')
            for film in liked_films:
                print(film)

            #asking the user to validate the selected list of films
            correct = input("Please verify that the above titles are correct y/n:\n")
        
        else:
            correct = 'n'
            print('Error, please try again')
            
#creating the list of suggestions for the three selected titles

    #getting the unique ids for the liked movies:
    liked_films_tconst = []
    for film in liked_films:
        liked_films_tconst.append(\
            movie_ids[movie_ids['full_title'] == film]['tconst'].values[0])
    
    #getting the top 20 recommended films for each liked movie based on reviews
    rec_list_1 = list(review_recommender.sort_values(\
                                liked_films_tconst[0])['tconst'][1:21])
    rec_list_2 = list(review_recommender.sort_values(\
                                liked_films_tconst[1])['tconst'][1:21])
    rec_list_3 = list(review_recommender.sort_values(\
                                liked_films_tconst[2])['tconst'][1:21])
    
    #translating the tconst back to titles
    rec_list_1_titles = []
    rec_list_2_titles = []
    rec_list_3_titles = []
    
    for tconst in rec_list_1:
        rec_list_1_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_2:
        rec_list_2_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_3:
        rec_list_3_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    #getting the list of recommendations based on product similarity
    
    #getting the top 20 recommended films for each liked movie
    rec_list_4 = list(cos_df.sort_values(liked_films_tconst[0], ascending=False)\
                      ['tconst'][1:21])
    rec_list_5 = list(cos_df.sort_values(liked_films_tconst[1], ascending=False)\
                      ['tconst'][1:21])
    rec_list_6 = list(cos_df.sort_values(liked_films_tconst[2], ascending=False)\
                      ['tconst'][1:21])
    
    #translating the tconst back to titles
    rec_list_4_titles = []
    rec_list_5_titles = []
    rec_list_6_titles = []
    
    for tconst in rec_list_4:
        rec_list_4_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_5:
        rec_list_5_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
    for tconst in rec_list_6:
        rec_list_6_titles.append(\
            list(movie_ids[movie_ids['tconst'] == tconst]['full_title'])[0])
        
#if there are films that are recommended based on more than 1 liked movie
    #prioritize these films
    final_recs = intersection(rec_list_1_titles, rec_list_2_titles)
    final_recs = intersection(final_recs, rec_list_3_titles)
    final_recs = intersection(final_recs, rec_list_4_titles)
    final_recs = intersection(final_recs, rec_list_5_titles)
    final_recs = intersection(final_recs, rec_list_6_titles)
    
    #only want to return 10 recommendations
    for i in range(0, 10):
        final_recs.extend([rec_list_1_titles[i],\
                            rec_list_2_titles[i],\
                            rec_list_3_titles[i],\
                            rec_list_4_titles[i],\
                            rec_list_5_titles[i],\
                            rec_list_6_titles[i]])
    
    #printing the recommendations
    for i in range(0,10):
        print(final_recs[i], '\n')
    
#     print(final_recs[0], '\n', 
#           final_recs[1], '\n',
#           final_recs[2], '\n',
#           final_recs[3], '\n',
#           final_recs[4]
#          )
    
    #asking the user to "rate" the films that we have recommended to see if we're doing well
    bad_format = 'y'
    
    while bad_format == 'y':
        film_reviews = input('Please rate the above films on how likely you are to watch them (1-5). Separate the scores with commas, \n')
        film_reviews = film_reviews.replace(' ','').split(',')
        for i in range(0, len(film_reviews)):
            film_reviews[i] = int(film_reviews[i])
        
         #checking to see if they have input their scores correctly
        if all(isinstance(x, int) for x in film_reviews) == True\
        and len(film_reviews) == 10\
        and min(film_reviews) >= 1\
        and max(film_reviews) <= 5:
            print('Thank You!')
            break
        else:
            bad_format = 'y'